In [1]:
import pandas as pd

## 1 - Reading the main dataframe

We have a dataset that contains information for different organizations and period of time (year & month). For the purpose of this project, I decided to only keep the 3 main columns for us: 'OrganizationalUnitID','Year' & 'Month'

Let's first read the CSV and have a look at it.

In [2]:
#read the csv

df = pd.read_csv("data.csv", low_memory=False)

In [3]:
df

,OrganizationalUnitID,Year,Month
0,18.0,2017.0,1.0
1,18.0,2017.0,2.0
2,18.0,2017.0,3.0
3,18.0,2017.0,4.0
4,18.0,2017.0,5.0
...,...,...,...
514275,1606.0,2022.0,2.0
514276,1606.0,2022.0,2.0
514277,1606.0,2022.0,1.0
514278,1606.0,2022.0,1.0


It looks like the 'Year' and 'Month' columns have float type.

In [4]:
#chech the dtypes

df.dtypes

OrganizationalUnitID    float64
Year                    float64
Month                   float64
dtype: object

Let's change the type for the 'Year' and 'Month' columns

In [5]:
#change the dtypes

df = df.astype({"OrganizationalUnitID": object,
                     "Year": object, 
                    "Month": object})

## 2 - Get the distinct OrganizationalUnitID (distinct locations)

Now I want to obtain the distinct OrganizationUnitID, which represent a unique location. So I need to groupby distinct OrganizationUnitID.

In [6]:
#groupby OrganizationalUnitID

distinct_orga = df.groupby(by=['OrganizationalUnitID']).count()

In [7]:
#only keep the OrganizationalUnitID and change the dtype

distinct_orga = distinct_orga.reset_index()
distinct_orga = distinct_orga[['OrganizationalUnitID']]
distinct_orga = distinct_orga.astype({"OrganizationalUnitID": object})

In [8]:
distinct_orga

,OrganizationalUnitID
0,16
1,18
2,55
3,81
4,83
...,...
469,1681
470,1682
471,1684
472,1685


In [9]:
#check the number of distinct locations

distinct_orga['OrganizationalUnitID'].nunique()

474

## 3 - Give a random number for each distinct OrganizationalUnitID

In order to assign a random location later on, I start to assign a random number for each distinct OrganizationalUnitID. The number will go from 1 to 30 because I will assign one of the 30 biggest city in Spain.

In [10]:
#for each rows random number from 1 to 30

import random
distinct_orga['number'] = [ random.randint(1,30)  for k in distinct_orga['OrganizationalUnitID']]

In [11]:
distinct_orga

,OrganizationalUnitID,number
0,16,9
1,18,8
2,55,10
3,81,1
4,83,27
...,...,...
469,1681,12
470,1682,21
471,1684,11
472,1685,15


## 4 - Reading the file containing the latitude & longitude of Spanish cities

In [12]:
geo = pd.read_csv("latitude_longitude.csv")

In [13]:
#each distinct location has a unique number

geo

,number,city,latitude,longitude
0,1,Madrid,40.4167,-3.7167
1,2,Barcelona,41.3825,2.1769
2,3,Sevilla,37.3900,-5.9900
3,4,Málaga,36.7194,-4.4200
4,5,Valencia,39.4700,-0.3764
...,...,...,...,...
2438,2439,Palencia,42.0167,-4.5333
2439,2440,Lleida,41.6167,0.6222
2440,2441,Segovia,40.9481,-4.1184
2441,2442,Ciudad Real,38.9833,-3.9167


In [14]:
distinct_orga

,OrganizationalUnitID,number
0,16,9
1,18,8
2,55,10
3,81,1
4,83,27
...,...,...
469,1681,12
470,1682,21
471,1684,11
472,1685,15


## 5 - Join the distinct_orga & geo dataframes

Now that I assigned a random number to each distinct OrganizationalUnitID and that the geo dataframe has been created (that contains a number (Id), city, latitude, and longitude), I can join both using the 'number' column.

In [15]:
distinct_orga = pd.merge(geo, distinct_orga, on='number', how='inner')

In [16]:
distinct_orga

,number,city,latitude,longitude,OrganizationalUnitID
0,1,Madrid,40.4167,-3.7167,81
1,1,Madrid,40.4167,-3.7167,108
2,1,Madrid,40.4167,-3.7167,212
3,1,Madrid,40.4167,-3.7167,260
4,1,Madrid,40.4167,-3.7167,266
...,...,...,...,...,...
469,30,Almería,36.8403,-2.4681,979
470,30,Almería,36.8403,-2.4681,1526
471,30,Almería,36.8403,-2.4681,1579
472,30,Almería,36.8403,-2.4681,1583


## 6 - Add the information to the main file for the different OrganizationalUnitID (latitude, longitude, city)

We can add the informations created on the main dataframe including the city, latitude, longitude & number

In [17]:
df = pd.merge(df, distinct_orga, on='OrganizationalUnitID', how='inner')

In [18]:
#drop the number column used to join

df.drop(columns='number', inplace=True)

In [19]:
df

,OrganizationalUnitID,Year,Month,city,latitude,longitude
0,18,2017,1,Murcia,37.9861,-1.1303
1,18,2017,2,Murcia,37.9861,-1.1303
2,18,2017,3,Murcia,37.9861,-1.1303
3,18,2017,4,Murcia,37.9861,-1.1303
4,18,2017,5,Murcia,37.9861,-1.1303
...,...,...,...,...,...,...
514190,1688,2021,3,Almería,36.8403,-2.4681
514191,1688,2021,7,Almería,36.8403,-2.4681
514192,1688,2021,7,Almería,36.8403,-2.4681
514193,1688,2021,8,Almería,36.8403,-2.4681


Check the unique years

In [20]:
df['Year'].unique()

array([2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0, 2016.0, 2015.0,
       2014.0, 2013.0], dtype=object)

Check the unique months

In [21]:
df['Month'].unique()

array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0],
      dtype=object)

As we have some null, we can fill out them with NaN

In [22]:
import numpy as np

In [23]:
#fill the na with np.nan

df.fillna(np.nan)

,OrganizationalUnitID,Year,Month,city,latitude,longitude
0,18.0,2017.0,1.0,Murcia,37.9861,-1.1303
1,18.0,2017.0,2.0,Murcia,37.9861,-1.1303
2,18.0,2017.0,3.0,Murcia,37.9861,-1.1303
3,18.0,2017.0,4.0,Murcia,37.9861,-1.1303
4,18.0,2017.0,5.0,Murcia,37.9861,-1.1303
...,...,...,...,...,...,...
514190,1688.0,2021.0,3.0,Almería,36.8403,-2.4681
514191,1688.0,2021.0,7.0,Almería,36.8403,-2.4681
514192,1688.0,2021.0,7.0,Almería,36.8403,-2.4681
514193,1688.0,2021.0,8.0,Almería,36.8403,-2.4681


## 7 - Group by the main df by 'OrganizationalUnitID','Year','Month','latitude','longitude'

In [24]:
orga_year_month = df.groupby(by=['OrganizationalUnitID','Year','Month','latitude','longitude','city']).count()

In [25]:
orga_year_month = orga_year_month.reset_index()

orga_year_month = orga_year_month[['OrganizationalUnitID','Year','Month','latitude','longitude','city']]

In [26]:
orga_year_month = orga_year_month.astype({"OrganizationalUnitID": object,
                                          "Year": object, 
                                          "Month": object,
                                         "latitude": object,
                                         "longitude": object})

In [29]:
orga_year_month

,OrganizationalUnitID,Year,Month,latitude,longitude,city
0,16,2020,1,28.1272,-15.4314,Las Palmas
1,16,2020,2,28.1272,-15.4314,Las Palmas
2,16,2020,3,28.1272,-15.4314,Las Palmas
3,18,2017,1,37.9861,-1.1303,Murcia
4,18,2017,2,37.9861,-1.1303,Murcia
...,...,...,...,...,...,...
9501,1688,2021,8,36.8403,-2.4681,Almería
9502,1688,2021,9,36.8403,-2.4681,Almería
9503,1688,2021,10,36.8403,-2.4681,Almería
9504,1688,2021,11,36.8403,-2.4681,Almería


## 8 - Weather extraction

Using the Meteostat library, we can extract the monthly weather information for the distinct OrganizationalUnitID, Year, Month taking into consideration the location (Latitude & Longitude)

More specifically, we can ingest the following indicators:
- tavg: The mean air temperature in °C
- tmin: The mean minimum air temperature in °C
- tmax: The mean maximum air temperature in °C
- prcp: The mean monthly precipitation total in mm
- wspd: The mean wind speed in km/h
- tsun: The mean sunshine total in minutes (m)

In [30]:
from numpy.random import seed
seed(42)
from datetime import datetime
from meteostat import Point, Monthly
from meteostat import Stations

I define the ingestion and cleaning in one main function:

In [31]:
def extract_weather(df):
    #initiate empty lists for the different indicators
    indexes=[]
    tavg=[]
    tmin=[]
    tmax=[]
    prcp=[]
    wspd=[]
    tsun=[]
    original_data = df[['longitude','latitude','Year','Month','OrganizationalUnitID']]
    for index, row in original_data.iterrows():
        #start is the first year, month and day of the reference period
        start = datetime(int(row['Year']), int(row['Month']), 1)
        
        #end is the last year, month and day of the reference period
        end = datetime(int(row['Year']), int(row['Month']), 1)
        
        #city is defined using the latitude and longitude
        city = Point(float(row['latitude']),float(row['longitude']))
        
        #we assemble all the row information: city, start day and end day
        #in our situation start = end
        weather_data = Monthly(city, start, end)
        weather_data = weather_data.fetch()
        weather_data = weather_data.iloc[:1].reset_index()
        
        #append all the unique indexes in index list
        indexes.append(index)
        
        #if we have no information for tavg for the row we put NaN, else we append to the metric list.
        if not np.any(weather_data['tavg']):
            tavg.append(np.NaN)
        else:  
            tavg.append(weather_data['tavg'][0])
            
        #if we have no information for tmin for the row we put NaN, else we append to the metric list.
        if not np.any(weather_data['tmin']):
            tmin.append(np.NaN)
        else:  
            tmin.append(weather_data['tmin'][0])
            
        #if we have no information for tmax for the row we put NaN, else we append to the metric list.
        if not np.any(weather_data['tmax']):
            tmax.append(np.NaN)
        else:  
            tmax.append(weather_data['tmax'][0])
            
        #if we have no information for prcp for the row we put NaN, else we append to the metric list.
        if not np.any(weather_data['prcp']):
            prcp.append(np.NaN)
        else:  
            prcp.append(weather_data['prcp'][0])
        
        #if we have no information for wspd for the row we put NaN, else we append to the metric list.
        if not np.any(weather_data['wspd']):
            wspd.append(np.NaN)
        else:  
            wspd.append(weather_data['wspd'][0])
            
        #if we have no information for tsun for the row we put NaN, else we append to the metric list.
        if not np.any(weather_data['tsun']):
            tsun.append(np.NaN)
        else:  
            tsun.append(weather_data['tsun'][0])
            
    #create a tuple including the list for all the meatrics
    weather_tuples = list(zip(indexes,tavg,tmin,tmax,prcp,wspd,tsun))
    
    #from this tuple we create a df that indicates for each id the metrics
    weather_df = pd.DataFrame(weather_tuples, columns=['id','tavg','tmin','tmax','prcp','wspd','tsun'])
    return weather_df

In [32]:
#create our weather_df from our df

weather_df = extract_weather(orga_year_month)

In [33]:
weather_df

,id,tavg,tmin,tmax,prcp,wspd,tsun
0,0,17.8,14.7,20.8,2.0,14.7,14700.0
1,1,20.1,16.9,23.4,1.0,19.5,15300.0
2,2,18.8,15.8,21.9,16.0,NaN,14880.0
3,3,10.1,5.1,16.2,50.8,NaN,NaN
4,4,13.3,8.2,19.4,3.8,NaN,NaN
...,...,...,...,...,...,...,...
9501,9501,27.6,23.8,31.4,0.5,13.5,18000.0
9502,9502,26.0,21.8,30.2,0.5,14.2,16380.0
9503,9503,21.8,17.8,25.8,27.0,15.4,15180.0
9504,9504,15.6,11.7,20.4,5.8,17.0,NaN


We can check the number of nulls for every metric

In [34]:
weather_df.isna().sum()

id         0
tavg     546
tmin     546
tmax     542
prcp     851
wspd    1518
tsun    2855
dtype: int64

Look at the main statistics

In [35]:
weather_df.describe()

,id,tavg,tmin,tmax,prcp,wspd,tsun
count,9506.000000,8960.000000,8960.000000,8964.000000,8655.000000,7988.000000,6651.000000
mean,4752.500000,16.365737,11.406607,21.545906,51.400289,11.421545,13860.207488
std,2744.290163,6.037415,5.797510,6.519323,58.276547,4.171277,4838.863026
min,0.000000,2.400000,-2.100000,7.200000,0.100000,2.400000,2340.000000
25%,2376.250000,11.800000,6.900000,16.600000,12.000000,8.600000,10260.000000
50%,4752.500000,15.600000,10.700000,20.600000,36.000000,10.900000,13500.000000
75%,7128.750000,21.000000,15.800000,26.500000,67.750000,13.400000,17340.000000
max,9505.000000,30.700000,24.600000,39.800000,471.000000,43.900000,24720.000000


## 9 - Obtain the final file that contains every information

In [36]:
final_file_weather = pd.concat([weather_df, orga_year_month],axis=1)

In [37]:
final_file_weather

,id,tavg,tmin,tmax,prcp,wspd,tsun,OrganizationalUnitID,Year,Month,latitude,longitude,city
0,0,17.8,14.7,20.8,2.0,14.7,14700.0,16,2020,1,28.1272,-15.4314,Las Palmas
1,1,20.1,16.9,23.4,1.0,19.5,15300.0,16,2020,2,28.1272,-15.4314,Las Palmas
2,2,18.8,15.8,21.9,16.0,NaN,14880.0,16,2020,3,28.1272,-15.4314,Las Palmas
3,3,10.1,5.1,16.2,50.8,NaN,NaN,18,2017,1,37.9861,-1.1303,Murcia
4,4,13.3,8.2,19.4,3.8,NaN,NaN,18,2017,2,37.9861,-1.1303,Murcia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9501,9501,27.6,23.8,31.4,0.5,13.5,18000.0,1688,2021,8,36.8403,-2.4681,Almería
9502,9502,26.0,21.8,30.2,0.5,14.2,16380.0,1688,2021,9,36.8403,-2.4681,Almería
9503,9503,21.8,17.8,25.8,27.0,15.4,15180.0,1688,2021,10,36.8403,-2.4681,Almería
9504,9504,15.6,11.7,20.4,5.8,17.0,NaN,1688,2021,11,36.8403,-2.4681,Almería


In [38]:
final_file_weather.drop(columns='id', inplace=True)

In [39]:
final_file_weather

,tavg,tmin,tmax,prcp,wspd,tsun,OrganizationalUnitID,Year,Month,latitude,longitude,city
0,17.8,14.7,20.8,2.0,14.7,14700.0,16,2020,1,28.1272,-15.4314,Las Palmas
1,20.1,16.9,23.4,1.0,19.5,15300.0,16,2020,2,28.1272,-15.4314,Las Palmas
2,18.8,15.8,21.9,16.0,NaN,14880.0,16,2020,3,28.1272,-15.4314,Las Palmas
3,10.1,5.1,16.2,50.8,NaN,NaN,18,2017,1,37.9861,-1.1303,Murcia
4,13.3,8.2,19.4,3.8,NaN,NaN,18,2017,2,37.9861,-1.1303,Murcia
...,...,...,...,...,...,...,...,...,...,...,...,...
9501,27.6,23.8,31.4,0.5,13.5,18000.0,1688,2021,8,36.8403,-2.4681,Almería
9502,26.0,21.8,30.2,0.5,14.2,16380.0,1688,2021,9,36.8403,-2.4681,Almería
9503,21.8,17.8,25.8,27.0,15.4,15180.0,1688,2021,10,36.8403,-2.4681,Almería
9504,15.6,11.7,20.4,5.8,17.0,NaN,1688,2021,11,36.8403,-2.4681,Almería


In [40]:
final_file_weather.to_csv('weather_data.csv')